In [ ]:
from IPython.display import HTML, Javascript, display
from ipywidgets import Image
from PIL import Image as Image2
from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox, Button, Textarea, jslink, Text, VBox
import ipycanvas
from ipycanvas import Canvas, MultiCanvas, hold_canvas
from os import path
import time
from io import StringIO, BytesIO
from base64 import b64encode, b64decode
import numpy as np

In [ ]:
#'//div[@id="marker"]/../../..//canvas'
Javascript("""
window.find_element = function(xpath, i){
    let results = document.evaluate(
        xpath,
        document,
        null,
        XPathResult.ORDERED_NODE_SNAPSHOT_TYPE,
        null
    ).snapshotItem(i);
    return results;
};
""")

In [ ]:
# get ClipboardImage
#https://b-estack.com/2018/12/25/clipboard_api/
Javascript("""
window.read_clipboard = function(f){
    navigator.clipboard.read()
    .then(clipboardItems => {
        const clipboardItem = clipboardItems[0];
        let type = clipboardItem.types[0]
        if (type == "image/png"){
            clipboardItem.getType(type)
            .then(blob => {
                var reader = new FileReader();
                reader.onload = function(event){
                    f(event.target.result)
                };
                var source = reader.readAsDataURL(blob);
            })
            .catch(err => {
                console.error('blob error', err);
            });
        }
        else{
            console.error("unknown type");
        }
    })
    .catch(err => {
        console.error('permission denied', err);
    });
}
""")

In [ ]:
width = 256
height = 256
clipboard_layer = Canvas(width=width, height=height, sync_image_data=True)
clipboard_layer.fill_style = 'white'
clipboard_layer.fill_rect(0, 0, width, height)

marker = HTML('<div id="marker"></div>')
text_button = Button(description='clipboard→textarea')
canvas_button = Button(description='textarea→canvas')

def paste_canvas(b):
#    image = Image.from_url(text.value)
    image = Image2.open(BytesIO(b64decode(text.value.split(",")[-1])))
    src_width, src_height = image.size
    if (src_width > src_height):
        dst_width = width
        dst_height = int(height*src_height/src_width)
    else:
        dst_width = int(width*src_width/src_height)
        dst_height = height
    image = image.resize((dst_width, dst_height))
        
    with hold_canvas(clipboard_layer):
        clipboard_layer.fill_style = 'white'
        clipboard_layer.fill_rect(0, 0, width, height)
        clipboard_layer.put_image_data(
            np.asarray(image),
            int(width/2-dst_width/2),
            int(height/2-dst_height/2)
        )
canvas_button.on_click(paste_canvas)

text = Text()
app = AppLayout(
    center=VBox((
        HBox((text_button, canvas_button, text)),
        clipboard_layer
    )),
)
display(marker,  app)

In [ ]:
Javascript("""
let text = find_element('//div[@id="marker"]/../../..//input', 0);
let button = find_element('//div[@id="marker"]/../../..//button', 0);
button.onclick = function() {
    read_clipboard(function(b){
        text.value = b;
        let event = new Event('change', {bubbles: true});
        text.dispatchEvent(event);
    });
}
""")